# Differentiable Programming for Two-Phase Flow Model in Porous Media with Barrier Effect

### Gregory Dushkin (MIPT), Roland Grinis (MIPT, GrinisRIT)

### Diffusion type evolution equations

In this section, we consider a non-stationary diffusion type system of PDEs in general form relevant for modelling multi-physics multi-phase flow in heterogeneous media with barrier effects, as in:

* Fučík, R. et. al. , Multidimensional mixed–hybrid finite element method for compositional two-phase flow in heterogeneous porous media and its parallel implementation on GPU. *Computer Physics Communications*, 238, 2019.

Exact expressions for the coefficients depend on the physics and the modelling set-up. 

Let $\boldsymbol{Z}=\boldsymbol{Z}(t,x; \theta) \in \mathbb{R}^{n}$ 
denote the primary unknown variables over a space-time domain
$(t,x) \in \left[0,T\right] \times \varOmega \subset \mathbb{R}^{1+d}$ 
and control parameters 
$\theta \in \mathbb{R}^p$. 
We pose $\varOmega$ a polygonal 
$d$-dimensional domain (typically $d=3$) and $T$ finite simulation time.

Introduce the Darcy velocity vector fields:
$$
\boldsymbol{v}_i = - \sum_{j=1}^{n} \boldsymbol{D}_{ij} \nabla_x Z_j +  \boldsymbol{w}_i
$$
for a symmetric positive-definite diffusion tensor $\boldsymbol{D}$ and external conservative force field $\boldsymbol{w}$. 

By convention we denote tensors in bold as opposed to their scalar components, e.g. $\boldsymbol{Z} = \left( Z_i \right)_{i=1}^n$, but $\boldsymbol{D} = \left( \boldsymbol{D}_{ij} \right)_{i,j=1}^n$. They can depend on space-time coordinates, the variables $\boldsymbol{Z}$ and the parameters $\theta$.

Multiplying $\boldsymbol{v}$ by the mobility coefficients $\boldsymbol{m}$, we obtain the conservative flux:
$$
\boldsymbol{q}_i = m_i\boldsymbol{v}_i 
$$
  
The main system governing the evolution is given by:
$$
\sum_{j=1}^{n}\left( N_{ij} \partial_t + \boldsymbol{u}_{ij} \cdot \nabla + r_{ij} \right) Z_j + \nabla_x \cdot \left( \boldsymbol{q}_i + \sum_{j=1}^{n} Z_j \boldsymbol{a}_{ij} \right) = f_i 
$$
where $\boldsymbol{N}$ is the damping matrix, $\boldsymbol{r}$ the reaction matrix, $\boldsymbol{a}$ and $\boldsymbol{u}$ the convection vector fields, and finally $\boldsymbol{f}$ the source or sink term.

Moreover we have:
* $\boldsymbol{Z}(0,x; \theta) = \boldsymbol{Z}^{\mathcal{I}}(x; \theta)$, $\forall x \in \varOmega$ (initial conditions)
* $Z_i(t,x; \theta) = Z_i^{\mathcal{D}}(t,x;\theta)$, $\forall x \in \varGamma_{Z_i} \subset \partial\varOmega$ (Dirichlet boundary conditions)
* $\boldsymbol{q}_i \cdot \boldsymbol{n} = q_i^{\mathcal{N}}(\theta)$, $\forall x \in \varGamma_{\boldsymbol{q}_i} \subset \partial\varOmega$ with $\boldsymbol{n}$ outward unit normal to $\partial\varOmega$ (Neumann boundary conditions)

The function $\boldsymbol{Z}$ is required to be continuously differentiable in time and only continuous in space. It satisfies the evolution equations in the weak sense. We discuss the numerical scheme to solve this system of PDEs below.

In applications, we are typically interested to determine the parameter $\theta$ such that we match a given measurement (inverse problem) or minimise a loss function (optimisation) that we express through a functional 
$$
\mathcal{L}(\theta) = \mathcal{L}\left[\boldsymbol{Z}(\theta)\right]
$$ 

To tackle this type of problems efficiently, especially for higher dimensional parameter space $p \gg 1$, we need to be able to compute at least the first order sensitivity:
$$
\nabla_{\theta} \mathcal{L} =  \nabla_{\boldsymbol{Z}}\mathcal{L} \cdot \nabla_{\theta}\boldsymbol{Z} 
$$

However, computing $\nabla_{\theta}\boldsymbol{Z}$ is impractical in most scenarios. After discretisation, the primary variables will be represented by a vector of high-dimension $N \gg 1$ and the gradient ends up being a dense $N \times p$ matrix, which is not scalable to large problems. Moreover, it might not exist for all time $t \in \left[0,T\right]$. The linearized adjoint PDE typically exhibit stiffness or even singular behaviour, and designing a stable numerical solver for such systems is a challenge \cite{chris}.

We present an algorithm based on the backward discrete adjoint sensitivity method to evaluate $\nabla_{\theta} \mathcal{L}$ with the same complexity as for the construction of the solution $\boldsymbol{Z}$ itself, at the expense of linear memory consumption in time. For the specific example of breakthrough monitoring, the sensitivity of the solution will be non-trivial only for a short period of time and such memory complexity won't be of great concern to us. We will also describe in that specific example how to deal with the time slices for which the sensitivity is discontinuous.  

### Mixed-hybrid finite element method

The solver relies on a semi-implicit MHFEM scheme and benefits from \textsf{GPU} acceleration. It achieves comparable accuracy with the fully implicit box schemes for two-phase flow models in 2D and 3D, yet yields an order of magnitude speed-up already on CPU. We will give here an overview of the method.

Denote by $\mu_d$ the $d$-dimensional Lebesgue measure. Let $\mathcal{K}_h$ be a conformal mesh for the polygonal spatial domain $\varOmega$ with:
$$ 
\mu_d(K) = \mathcal{O}(h^d), \quad \forall K \in \mathcal{K}_h
$$ 
Define $\mathcal{V}_h$ to be the set of vertices and $\mathcal{F}_h$ the set of ($d-1$)-dimensional faces composing $\mathcal{K}_h$. Let us write $\mathcal{F}_K$ for the set of faces forming an element $K$.

We approximate the primary unknown variables $\boldsymbol{Z}$ to order zero, computing the spatial averages of $\boldsymbol{Z}$ for an element $K \in \mathcal{K}_h$ and a face $E \in \mathcal{F}_K$:
$$
\boldsymbol{Z}_K(t;\theta) = \frac{1}{\mu_d(K)} \int_{K}\boldsymbol{Z}(t,x;\theta)d\mu_{d,x}, 
$$
$$
\boldsymbol{Z}_E(t;\theta) = \frac{1}{\mu_{d-1}(E)} \int_{E}\boldsymbol{Z}(t,x;\theta)d\mu_{d-1,x}
$$
respectively. Discretising time $\{ t_k \}_{k=0}^N \subset [0,T]$ with typically adaptive time-steps, we denote time-slice restrictions by $\boldsymbol{Z}^k = \boldsymbol{Z}(t_k)$. 
We should write
$\boldsymbol{Z}_{\mathcal{K}_h}^k=\{ \boldsymbol{Z}_K^k\}_{K \in \mathcal{K}_h}$ and
$\boldsymbol{Z}_{\mathcal{F}_h}^k =\{ \boldsymbol{Z}_F^k\}_{F \in \mathcal{F}_h}$
for the vectors of element and face averages slices. Those are essentially our unknowns to solve for in the discretised system.

We recall that the lowest-order Raviart–Thomas–Nédélec space $\textbf{RTN}_0(K)$ is spanned by piece-wise linear vector fields $\omega_{K,E}$ for each $E \in \mathcal{F}_K$ whose flux is non-zero only through $E$:
$$
\omega_{K,E} \cdot \boldsymbol{n}_{K,F} = \frac{\delta_{EF}}{\mu_{d-1}(E)}
$$
and its divergence fixed to $\nabla \cdot \omega_{K,E} = \mu_d(K)^{-1}$.

Applying finite element methods to the PDE system in previous section over $\bigotimes_{K \in \mathcal{K}_h}\textbf{RTN}_0(K)$ in spatial directions, forward finite differences in time direction, gives the semi-implicit scheme:
$$ 
\boldsymbol{Z}_{\mathcal{K}_h}^{k+1} = \boldsymbol{A}^k \boldsymbol{Z}_{\mathcal{F}_h}^{k+1} + \boldsymbol{R}^k
$$
and linear constraint equation for the face averages:
$$ 
\boldsymbol{M}^k \boldsymbol{Z}_{\mathcal{F}_h}^{k+1} = \boldsymbol{b}^k
$$
produced from he mass balance equations and barrier conditions on internal sides, together with boundary conditions on external sides.

More details about the construction of the matrices can be below for our specific model. Now, it suffices to note that the coefficients depend on the averages $\boldsymbol{Z}_{\mathcal{K}_h}^k$ only, which leads to an efficient evaluation of $\nabla_{\boldsymbol{Z}_{\mathcal{K}_h}^k}\boldsymbol{Z}_{\mathcal{K}_h}^{k+1}$ by adjoint methods. Furthermore, although we deal with a sparse linear system, for every given element $K \in \mathcal{K}_h$ only the constituent faces $F \in \mathcal{F}_K$ are involved, and only adjacent elements are considered. Hence one could design, in principle, an implementation benefiting from GPU acceleration.

The matrix $\boldsymbol{M}^k$ is non-singular given that the diffusion tensors $\boldsymbol{D}$ are strictly positive definite. The solution to the system can be obtained via various GMRES type algorithms, but a mass-lumping technique is required to stabilise the scheme.

### Two-phase flow in porous media

The pressures $(p_w,p_n)$ for the wetting and non-wetting phases are primary variables. 

Effective saturations:
$$
S_{w,e} = \frac{S_w - S_{w,r}}{1-S_{w,r}}, \quad S_w+S_n=1
$$
are related to the capillary pressure:
$$
p_c = p_n - p_w
$$
via the Brooks & Corey model:
$$
S_{w,e}(p_c) = \left( \frac{p_c}{p_{BC}} \right)^{-\lambda_{BC}}, \quad p_c \geq p_{BC}
$$
where $p_{BC}$ is the *entry pressure* characteristic of the medium.

Relative permeability is modelled via Burdine:
$$
k_{r,w} = S_{w,e}^{3+\frac{2}{\lambda_{BC}}} \\
k_{r,n} = (1-S_{w,e})^2 \left( 1 - S_{w,e}^{1+\frac{2}{\lambda_{BC}}} \right)
$$

The mobility coefficients are given by:
$$
m_{\alpha} = \frac{k_{r,\alpha}}{\mu_{\alpha}}
$$
with $\mu_{\alpha}$ the $\alpha$-phase dynamic viscosity.

Darcy velocities are:
$$
\boldsymbol{v}_\alpha = - \kappa \left( \nabla p_\alpha - \rho_\alpha \boldsymbol{g} \right)
$$
and the flux:
$$
\boldsymbol{q}_{\alpha} = m_\alpha \boldsymbol{v}_\alpha
$$

### Barrier effect

At the interface between medium $I$ and $II$, while $p_c^I < p_{BC}^{II}$:
$$
\boldsymbol{q}_n^{I} = 0, \quad p_c^{II} = p_{BC}^{II}
$$
and then we gain continuity once $p_c^I=p_c^{II}$ is reached.

### Double-porosity
For porous $p_\alpha^1$ and fracture $p_\alpha^2$ continua pressures:
$$
\sum_{\beta=n,w} N_{\alpha\beta}^i \partial_t  p_\beta^i + \nabla \cdot \boldsymbol{q}_\alpha^i = - \frac{\chi k_{r,\alpha}^1}{\mu_\alpha^1} \left(p_\alpha^i - p_\alpha^j\right)
$$

where the damping matrix is given:
$$
\boldsymbol{N}^i = -\eta^i \frac{\partial S_w^i}{\partial p^i_c} 
\begin{pmatrix}
-1 & 1 \\
1 & -1
\end{pmatrix}
$$

The geometric shape parameter 
$$
\chi \sim  \frac{1}{\ell^2}
$$
depends on the characteristic fractures length $\ell$.

### Boundary conditions

We have:
* $p(0,x) = p^{\mathcal{I}}(x)$, $\forall x \in \varOmega$ (initial conditions)
* $p(t,x) = p^{\mathcal{D}}(t,x)$, $\forall x \in \varGamma_{Z_i} \subset \partial\varOmega$ (Dirichlet boundary conditions)
* $\boldsymbol{q}(t,x) \cdot \boldsymbol{n} = q^{\mathcal{N}}(t,x)$, $\forall x \in \varGamma_{\boldsymbol{q}_i} \subset \partial\varOmega$ with $\boldsymbol{n}$ outward unit normal to $\partial\varOmega$ (Neumann boundary conditions)

The pressures functions $p$ are required to be continuously differentiable in time and only continuous in space. The evolution equations are satisfied in the weak sense.

### Mass lumping

For one-phase flow Younes et al. use the mass lumping procedure by replacing edge-wise flux (flux leaving the cell $K$ through edge $E$) with the following approximation:
$$
Q_{K,E} =
\bar Q_{K,E} +
\frac { \mu_d \left( K \right) }{ n_f } f_K -
c \frac{ \mu_d \left( K \right) } {n_f} \frac{ \partial Z_{K,E} }{\partial t}
$$
where $\bar Q_{K,E}$ is the flux corresponding to stationary problem without sink/source terms and $n_f$ is the total amount of edges of element $K$.

In our case of two-phase flow damping matrix $\boldsymbol{N}$ takes place of $c$.

$Q_{K,E}$ is also representable in terms of velocity $Q_{i,K,E} = m_{i,K,E} v_{i,K,E}$, which means that we will approximate $\boldsymbol v_{K,E}$ as
$$
v_{i,K,E} = \bar v_{i,K,E} + \frac{ \mu_d \left( K \right) }{ m_i n_f } \left[ f_{i,K} - \sum\limits_{j=1}^2 N_{i,j,K} \frac{ \partial Z_{i,K,E} }{ \partial t } \right]
$$

Fučík et al.:
$$
v^k_{i,K,E} = \sum\limits_{j \in \sigma_{i,K}} \left(
    b_{i,j,K,E} Z^k_{j,K} - \sum\limits_{F \in \mathcal{F}_K} b_{i,j,K,E,F} Z_{j,F}^k
\right) + w_{i,K,E}
$$

$\sigma_{i,K}$ deontes the set of indices $j$ for which $\boldsymbol{D}_{i,j}$ is non-zero on element $K \in \mathcal{K}_h$.
In our case
$$
\boldsymbol{D} = \left( \begin{matrix}
    \lambda_t K \boldsymbol{I} & 0 \\
    0 & \lambda_t K \boldsymbol{I}
\end{matrix}\right)
$$
so
$$
\sigma_{i,K} = \left\{ i \right\}
$$
and
$$
v^k_{i,K,E} = b_{i,i,K,E}Z^k_{i,K} - \sum\limits_{F \in \mathcal{F}_K}b_{i,i,K,E,F}Z_{i,F}^k + w_{i,K,E}.
$$

$$
\frac{ \mu_d\left(K\right) }{ \tau } \sum\limits_{j=1}^2 N_{i,j,K} \left(Z_{j,K}^{k+1} - Z_{j,K}^k \right) + \sum\limits_{E \in \mathcal{F}_K} m_{i,E}^k v^{k + 1}_{i,K,E} = \mu_d\left(K\right) f_{i,K}
$$

In vector form:
$$
\frac{\|K\|}{\tau} \boldsymbol N_K \left( \boldsymbol Z_K^{k+1} - \boldsymbol Z_K^k \right) +  \boldsymbol m_K^k\boldsymbol v_K^{k+1} = \|K\|\boldsymbol f_K
$$

$$
\boldsymbol v_K^{k+1} = \boldsymbol b_{\Sigma, K} \boldsymbol Z^{k+1}_K - \boldsymbol b_K \boldsymbol Z^{k+1}_{F,K} + \boldsymbol w_K
$$

$$
\boldsymbol v_K = \left(\begin{matrix}
    v_{1,E_1,K} \\ v_{1,E_2,K} \\ ... \\ v_{2,E_1,K} \\ v_{2,E_2,K} \\ ...
\end{matrix}\right)\text{ }\left(\boldsymbol w_K\text{ and }\boldsymbol Z_{F,K}\text{ are similar}\right)
$$

$$
\boldsymbol m_K = \left(\begin{matrix} m_{1,E_1} & m_{1,E_2} & ... & 0 & 0 & ... \\ 0 & 0 & ... & m_{2,E_1} & m_{2,E_2} & ... \end{matrix}\right)\text{ }\left(\boldsymbol b_K\text{ is similar}\right)
$$

$$\boldsymbol b_{\Sigma,K} = \left(\begin{matrix} b_{1,1,K,E_1} & 0 \\ b_{1,1,K,E_2} & 0 \\ ... & ... \\ 0 & b_{2,2,K,E_1} \\ 0 & b_{2,2,K,E_2} \\ ... & ... \end{matrix}\right)$$

$$
\frac{\|K\|}{\tau}\boldsymbol N_K \left( \boldsymbol Z_K^{k+1} - \boldsymbol Z_K^k \right) + \boldsymbol m_K^K \boldsymbol b_{\Sigma,K} \boldsymbol Z_K^{k+1} - \boldsymbol m_K^k \left( \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} + \boldsymbol w_K \right) = \|K\| \boldsymbol f_K
$$

$$
\left( \frac{\|K\|}{\tau} \boldsymbol N_K + \boldsymbol m_K^k \boldsymbol b_{\Sigma,K} \right) \boldsymbol Z_K^{k+1}
= \|K\|\boldsymbol f_K + \frac{\|K\|}{\tau}\boldsymbol N_K\boldsymbol Z_K^k + \boldsymbol m_K^k \left( \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} + \boldsymbol w_K \right)
$$

Let's denote $\Lambda^k_K = \left( \frac{\|K\|}{\tau} \boldsymbol N_K + \boldsymbol m_K^k\boldsymbol b_{\Sigma,K} \right)$.
It's a square $2 \times 2$ matrix and, for solution to exist, it must be nonsingular. Then
$$
\boldsymbol Z_K^{k+1} = \left(\boldsymbol \Lambda_K^k\right)^{-1} \left[
    \|K\|\boldsymbol f_K + \frac{\|K\|}{\tau} \boldsymbol N_K \boldsymbol Z_K^k + \boldsymbol m_K^k \left(
        \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} + \boldsymbol w_K
    \right)
\right].
$$

Then
$$
\boldsymbol v_K^{k+1} = \boldsymbol b_{\Sigma,K} \left( \boldsymbol \Lambda_K^k \right)^{-1} \left[
    \|K\|\boldsymbol f_K + \frac{\|K\|}{\tau} \boldsymbol N_K \boldsymbol Z_K^k + \boldsymbol m_K^k \left(
        \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} + \boldsymbol w_K
    \right)
\right] - \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} + \boldsymbol w_K.
$$

Writing mass balance equation for $v_{i,K,E}^k$ then helps obtain the finla system for $\boldsymbol Z_{F,K}^{k+1}$.

Following Younes et al. let's write $\boldsymbol v_K^{k+1}$ for stationary case with no sink/source ($\boldsymbol N =0$, $\boldsymbol f$ = 0, and also $boldsymbol w = 0$, which for this purpose could be considered a part of free-term):
$$
\bar{ \boldsymbol v }_K^{k+1} = \boldsymbol b_{\Sigma,K} \left(
    \boldsymbol m_K^k\boldsymbol b_{\Sigma,K}
\right)^{-1} \boldsymbol m_K^k \left( \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} \right) - \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1}
$$

And perform mass lumping by adding to $\bar v_{i,K,E}^k$ the term
$$
\frac{\|K\|}{n_f} \left( f_{i,K} - \sum\limits_{E \in \mathcal F_K} m_{i,E}^k w_{i,K,E} - \sum\limits_{j=1}^2 N_{i,j,K} \frac{ \partial Z_{i,E}}{\partial t} \right)
$$

or in vector form
$$
\frac{\|K\|}{n_f} \left[
    \left(
        \boldsymbol f_K - \boldsymbol m_K^k \boldsymbol w_K - \frac{1}{\tau} \bar{ \boldsymbol N }_K \left(
            \boldsymbol Z_{F,K}^{k+1} - \boldsymbol Z_{F,K}^k
        \right)
    \right)
\right]
$$

where $$
\bar{ \boldsymbol N }_K = \left(\begin{matrix}
    N_{1,1,K} & 0 & ... & N_{1,2,K} & 0 & ... \\
    0 & N_{1,1,K} & ... & 0 & N_{1,2,K} & ... \\
    ... & ... & ... & ... & ... & ... \\
    N_{2,1,K} & 0 & ... & N_{2,2,K} & 0 & ... \\
    0 & N_{2,1,K} & ... & 0 & N_{2,2,K} & ... \\
    ... & ... & ... & ... & ... & ...
\end{matrix}\right)
$$

This way the new, lumped system is obtained by writing mass balance for
$$
\boldsymbol v_K^{k+1} = \left( \boldsymbol b_{\Sigma,K} \left( \boldsymbol m_K^k \boldsymbol b_{\Sigma, K} \right)^{-1} \boldsymbol m_K^k - \boldsymbol I \right) \boldsymbol b_K \boldsymbol Z_{F,K}^{k+1} + \frac{\|K\|}{n_f} \left[
    \boldsymbol f_K - \boldsymbol m_K^k \boldsymbol w_K - \frac{1}{\tau} \bar{ \boldsymbol N_K } \left( \boldsymbol Z_{F,K}^{k+1} - \boldsymbol Z_{F,K}^k \right)
\right]
$$

<font color=red>
As far as I can tell
$$
    b_{\Sigma,K} ( m_K^k b_{\Sigma,K} )^{-1} m_K^k = I
$$

Am I wrong? Is the first term supposed to just disappear?!
</font>